# O projeto é uma análise de dados que deve conter as seguintes etapas:

•	Carregar e ler arquivos .csv, xlsx ou de um banco de dados;

•	Avaliar a necessidade da transformação da variável em outra escala (agrupar idade por faixas, por exemplo);

•	Realizar a codificação das variáveis categóricas conforme os valores das variáveis (label encoder, one hot encoder ou target encoder);

•	Normalizar as variáveis;

•	Avaliar a necessidade de realizar o balanceamento da variável alvo;

•	Tratar variáveis com alta correlação;

•	Realizar a seleção de variáveis;

•	Aplicar um modelo de regressão ou classificação utilizando uma técnica de hiperparametrização automática;

•	Aplicar uma medida de avaliação do modelo.

•	Gerar uma visualização para o resultado conforme os tipos abaixo:
    - Agrupamento: gerar a visualização Scatter;
    - Classificação: gerar a matriz de confusão; 
    - Regressão: gerar a visualização da linha de saída.


In [1]:
import pandas as pd # O 'as' é um alias, um apelido. Ao invés de escrever 'pandas', escrevemos 'pd'
import numpy as np

# 1. Carregar o arquivo .csv usando o pd.read_csv()

In [2]:
df = pd.read_csv('Pokemon.csv') # Lê o arquivo .csv e armazena em um DataFrame
df.head() #  Traz os 5 primeiros registros do DataFrame

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


# 2. Avaliar a necessidade da transformação da variável em outra escala (agrupar idade por faixas, por exemplo)

Não vimos necessidade de escalar as variáveis, pois o dataset é pequeno e autoexplicativo.

In [3]:
df.drop(['#'], axis=1, inplace=True) # Remove a coluna '#'; o axis 1 indica que é uma coluna, se fosse 0, indicaria que é uma linha; inplace=True indica que a alteração é feita no próprio DataFrame
colunas = ['nome', 'tipo_1', 'tipo_2', 'total', 'hp', 'ataque', 'defesa', 'sp_atq', 'sp_def', 'velocidade', 'geracao', 'lendario'] # traduzindo as colunas e renomeando de uma forma mais amigável para o modelo, tirando espaços e maiúsculas
df.columns = colunas # Renomeia as colunas do DataFrame
df.head()

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


# 3. Realizar a codificação das variáveis categóricas de acordo com os valores das variáveis (label encoder, one hot encoder ou target encoder)

In [4]:
# Label Encoder

from sklearn.preprocessing import LabelEncoder # Importa a classe LabelEncoder do módulo preprocessing da biblioteca sklearn

le = LabelEncoder() # O LabelEncoder faz sentido quando a variável categórica é ordinal, ou seja, tem uma ordem natural. O pokemon tem mais 'peso' por ser lendário.

df['lendario'] = le.fit_transform(df['lendario']) # Transforma os valores da coluna lendario em valores numéricos [0-1]
df.head()

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,0
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,0
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,0
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,0
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,0


# Normalizar as variáveis

In [14]:
# Antes, uma análise descritiva do DataFrame, é bom para observar valores discrepantes, outliers, e ter uma noção da distribuição dos dados.
df.describe()

,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
count,8.000000e+02,8.000000e+02,8.000000e+02,8.000000e+02,8.000000e+02,8.000000e+02,8.000000e+02,800.00000,800.00000
mean,-1.776357e-16,-2.486900e-16,5.329071e-17,-4.440892e-17,1.953993e-16,-1.421085e-16,-1.154632e-16,3.32375,0.08125
std,1.000626e+00,1.000626e+00,1.000626e+00,1.000626e+00,1.000626e+00,1.000626e+00,1.000626e+00,1.66129,0.27339
min,-2.127839e+00,-2.674852e+00,-2.281379e+00,-2.209039e+00,-1.920993e+00,-1.866223e+00,-2.178804e+00,1.00000,0.00000
25%,-8.766721e-01,-7.546915e-01,-7.399327e-01,-7.650653e-01,-7.054650e-01,-7.875334e-01,-8.015032e-01,2.00000,0.00000
50%,1.242618e-01,-1.668874e-01,-1.233542e-01,-1.232993e-01,-2.391303e-01,-6.840691e-02,-1.128526e-01,3.00000,0.00000
75%,6.664343e-01,4.209167e-01,6.473688e-01,5.184667e-01,6.782494e-01,6.507196e-01,7.479606e-01,5.00000,0.00000
max,2.876830e+00,7.278632e+00,3.421972e+00,5.010829e+00,3.705602e+00,5.684605e+00,3.846888e+00,6.00000,1.00000


In [6]:
# Antes, vamos tratar os NaN da coluna do tipo_2, pois muitos pokemons não tem 2 tipos de categorias.

df['tipo_2'].fillna('None', inplace=True) # Preenche os valores NaN da coluna tipo_2 com 'None'
df.isna().sum() # Verifica se há valores NaN no DataFrame

nome          0
tipo_1        0
tipo_2        0
total         0
hp            0
ataque        0
defesa        0
sp_atq        0
sp_def        0
velocidade    0
geracao       0
lendario      0
dtype: int64

In [7]:
df.head()

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,0
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,0
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,0
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,0
4,Charmander,Fire,None,309,39,52,43,60,50,65,1,0


In [8]:
from sklearn.preprocessing import StandardScaler # Importa a classe StandardScaler do módulo preprocessing da biblioteca sklearn

# O standardScaler é uma técnica de normalização que transforma os dados para que a média seja 0 e o desvio padrão seja 1.
# Se o valor da variável for menor que a média, o valor transformado será negativo. Se for maior, será positivo. Quanto maior o número, mais próximo, ou até maior que 1 ele será.

scaler = StandardScaler() # Instancia o StandardScaler
# Vamos usar o Scaler, pois ele é mais sensível a outliers, o que é comum em pokemons lendários. 

df_std = df[['total', 'hp', 'ataque', 'defesa', 'sp_atq', 'sp_def', 'velocidade']] = scaler.fit_transform(df[['total', 'hp', 'ataque', 'defesa', 'sp_atq', 'sp_def', 'velocidade']]) # Normaliza as colunas total, hp, ataque, defesa, sp_atq, sp_def e velocidade
df_std


array([[-0.97676549, -0.95062622, -0.92490618, ..., -0.2391303 ,
        -0.24818854, -0.80150318],
       [-0.25108843, -0.3628221 , -0.5241302 , ...,  0.21955954,
         0.29115635, -0.28501525],
       [ 0.74984544,  0.42091674,  0.09244823, ...,  0.83114599,
         1.01028289,  0.40363531],
       ...,
       [ 1.37542912,  0.42091674,  0.95565803, ...,  2.36011211,
         2.08897269,  0.05931003],
       [ 2.04271837,  0.42091674,  2.49710411, ...,  2.97169856,
         2.08897269,  0.40363531],
       [ 1.37542912,  0.42091674,  0.95565803, ...,  1.74852566,
         0.65071962,  0.05931003]])